In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline,Pipeline

In [113]:
df=pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [114]:
df.shape

(891, 12)

In [115]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [116]:
df.drop(["PassengerId","Name","Ticket","Cabin"],axis=1,inplace=True)

In [117]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [118]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=["Survived"]),df["Survived"],test_size=0.2,random_state=42)

In [119]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [120]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [121]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [65]:
#simple imputer to impute data
#si=SimpleImputer()
#df["Age"]=si.fit_transform(df[["Age"]])

In [109]:
#u can pass transformers on columns here
#ct=ColumnTransformer([("impute age",SimpleImputer(),[2]),("impute Embarked",SimpleImputer(strategy="most_frequent"),[6])])

In [110]:
#this will be my imputed data after using colummn transformer
#a=ct.fit_transform(x_train)

In [122]:
trf1=ColumnTransformer([("impute age",SimpleImputer(),[2]),("impute Embarked",SimpleImputer(strategy="most_frequent"),[6])],remainder='passthrough')

In [140]:
trf2=ColumnTransformer([("encoder",OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])],remainder="passthrough")

In [143]:
trf3=ColumnTransformer([("scaling",MinMaxScaler(),slice(0,10))])

In [130]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [132]:
trf5=DecisionTreeClassifier()

In [134]:
#creating pipelines:can be done using pipeline and also from make_pipeline

In [ ]:
#pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [144]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

In [146]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')


In [ ]:
#without model in pipe do fit transform

In [147]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute age', SimpleImputer(),
                                                  [2]),
                                                 ('impute Embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x0000029A79F4E790>)),
                ('trf5', DecisionTreeClassifier())])

In [151]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute age', SimpleImputer(), [2]),
                                 ('impute Embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('encoder',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                  slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x0000029A79F4E790>),
 'trf5': DecisionTreeClassifier()}

In [149]:
y_pred=pipe.predict(x_test)

In [150]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [152]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.6256983240223464

In [158]:
#crossvalidation using pipeline
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv=5, scoring='accuracy').mean()


0.6391214419383433

In [153]:
# gridsearchcv using pipeline
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}


In [155]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute '
                                                                         'age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute '
                                                                         'Embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('encoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scaling',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x0000029A79F4E790>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [156]:
grid.best_score_

0.6391214419383433

In [157]:
grid.best_params_

{'trf5__max_depth': 2}

In [159]:
#saving or exporting the pipeline
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

## Demo 2 Simple demo of pipeline

In [162]:
steps=[("scaling",MinMaxScaler()),("classifier",DecisionTreeClassifier())]

In [163]:
pipe=Pipeline(steps)

In [165]:
pipe.fit(x_train,y_train)

In [ ]:
y_pred=pipe.predict(x_test)

## Demo3

In [166]:
numeric_processor=Pipeline([("imputation",SimpleImputer(missing_values=np.nan)),("scaling",MinMaxScaler())])

In [167]:
numeric_processor

Pipeline(steps=[('imputation', SimpleImputer()), ('scaling', MinMaxScaler())])

In [168]:
categorical_processor=Pipeline([("imputation",SimpleImputer(missing_values="missing",strategy="constant")),("encoding",OneHotEncoder())])

In [169]:
categorical_processor

Pipeline(steps=[('imputation',
                 SimpleImputer(missing_values='missing', strategy='constant')),
                ('encoding', OneHotEncoder())])

In [170]:
preprocessor=ColumnTransformer([("categorical",categorical_processor,["name","city"]),("numerical",numeric_processor,["age","height"])])

In [171]:
pipe=make_pipeline(preprocessor,DecisionTreeClassifier())

In [172]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer(missing_values='missing',
                                                                                 strategy='constant')),
                                                                  ('encoding',
                                                                   OneHotEncoder())]),
                                                  ['name', 'city']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scaling',
                                                                   MinMaxScaler())]),
                                                  ['age', 'height'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])